# MNIST Genetic Algorithm
### ref : Deep Reinforcement Learning _in Action_ 

In [1]:
import warnings; warnings.simplefilter('ignore')
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
import torch
import torchvision.datasets as dset
from torch.distributions import Bernoulli
import torchvision.transforms as transforms
import numpy as np
import random
from matplotlib import pyplot as plt
from scipy.stats import halfnorm

Setup a directory to store the MNIST dataset/

In [3]:
root = './data'
if not os.path.exists(root):
    os.mkdir(root)

Setup a transformer to normalize the data.

In [4]:
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (1.0,))])

In [5]:
train_set = dset.MNIST(root=root, train=True, transform=trans, download=True)
test_set = dset.MNIST(root=root, train=False, transform=trans, download=True)

100.1%

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


113.5%

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw


100.4%

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


180.4%

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!


In [6]:
batch_size = 100

train_loader = torch.utils.data.DataLoader(
                 dataset=train_set,
                 batch_size=batch_size,
                 shuffle=True)
test_loader = torch.utils.data.DataLoader(
                dataset=test_set,
                batch_size=batch_size,
                shuffle=False)

We define a simple linear classifier (or you can think of it as a single layer neural network). It simply multiplies a weight/parameter matrix by the input vector and applies a softmax.

In [7]:
x = next(iter(train_loader))[0]

In [8]:
x = x.reshape(100,784)

In [9]:
class Individual:
    def __init__(self,param, fitness=0):
        self.param = param
        self.fitness = fitness

In [10]:
def model(x,W):
    return torch.nn.Softmax()(x @ W)

In [11]:
model(x,torch.rand(784,10))

tensor([[6.2397e-05, 4.3618e-02, 3.9825e-07, 3.7233e-05, 2.3996e-02, 3.0861e-01,
         8.9450e-05, 6.2265e-01, 7.6238e-05, 8.6209e-04],
        [2.4034e-05, 4.2048e-01, 2.3355e-03, 5.3658e-01, 1.9388e-04, 1.8655e-02,
         4.9889e-05, 2.1546e-02, 1.2826e-04, 5.4047e-06],
        [5.2195e-04, 1.8867e-01, 2.6529e-03, 3.6853e-03, 8.7694e-03, 7.5545e-01,
         1.1094e-03, 3.4726e-02, 2.3029e-03, 2.1122e-03],
        [1.7932e-03, 4.9951e-01, 1.8280e-02, 2.0288e-01, 8.4557e-02, 4.5482e-02,
         6.6736e-04, 1.3456e-01, 1.1781e-02, 4.8024e-04],
        [4.5312e-04, 7.3347e-01, 2.1193e-04, 3.8308e-03, 1.8762e-01, 3.8032e-03,
         7.0501e-04, 6.8948e-02, 2.2618e-04, 7.3552e-04],
        [3.2187e-02, 1.2166e-03, 5.9185e-07, 8.3157e-03, 3.4439e-04, 9.6708e-03,
         4.8334e-04, 9.4752e-01, 5.0564e-05, 2.1575e-04],
        [2.1903e-05, 1.0289e-03, 1.1728e-06, 4.1619e-05, 1.2273e-04, 6.3741e-04,
         7.8936e-04, 9.9735e-01, 2.5888e-06, 3.1801e-06],
        [8.0547e-04, 3.0429

In [12]:
def spawn_population(param_size=(784,10),pop_size=1000):
    return [Individual(torch.randn(*param_size)) for i in range(pop_size)]

In [13]:
loss_fn = torch.nn.CrossEntropyLoss()

In [14]:
random.randint(0,10)

4

In [15]:
def evaluate_population(pop):
    avg_fit = 0 #avg population fitness
    for individual in pop:
        x,y = next(iter(train_loader))
        pred = model(x.reshape(batch_size,784),individual.param)
        loss = loss_fn(pred,y)
        fit = loss
        individual.fitness = 1.0 / fit
        avg_fit += fit
    avg_fit = avg_fit / len(pop)
    return pop, avg_fit

In [17]:
# pop[0].param.shape

In [19]:
# torch.stack((pop[0].param.view(-1),pop[1].param.view(-1)),dim=0).view(-1).shape

In [20]:
def recombine(x1,x2): #x1,x2 : Individual
    w1 = x1.param.view(-1) #flatten
    w2 = x2.param.view(-1)
    cross_pt = random.randint(0,w1.shape[0])
    child1 = torch.zeros(w1.shape)
    child2 = torch.zeros(w1.shape)
    child1[0:cross_pt] = w1[0:cross_pt]
    child1[cross_pt:] = w2[cross_pt:]
    child2[0:cross_pt] = w2[0:cross_pt]
    child2[cross_pt:] = w1[cross_pt:]
    child1 = child1.reshape(784,10)
    child2 = child2.reshape(784,10)
    c1 = Individual(child1)
    c2 = Individual(child2)
    return [c1,c2]

In [21]:
def mutate(pop, mut_rate=0.01):
    param_shape = pop[0].param.shape
    l = torch.zeros(*param_shape)
    l[:] = mut_rate
    m = Bernoulli(l)
    for individual in pop:
        mut_vector = m.sample() * torch.randn(*param_shape)
        individual.param = mut_vector + individual.param
    return pop

In [22]:
def seed_next_population(pop,pop_size=1000, mut_rate=0.01):
    new_pop = []
    while len(new_pop) < pop_size: #until new pop is full
        parents = random.choices(pop,k=2, weights=[x.fitness for x in pop])
        offspring = recombine(parents[0],parents[1])
        new_pop.extend(offspring)
    new_pop = mutate(new_pop,mut_rate)
    return new_pop

In [23]:
pop = spawn_population()

In [25]:
%%time
pop, avg_fit = evaluate_population(pop)

CPU times: user 54.2 s, sys: 642 ms, total: 54.9 s
Wall time: 14.6 s


In [26]:
new_pop = seed_next_population(pop)

In [27]:
len(new_pop)

1000

Now we need to spawn a population of weight matrices, run the model using the different individuals, calculate the loss for each one, and then breed the ones with the highest fitness score (lowest loss).

In [28]:
num_generations = 50
population_size = 100
mutation_rate = 0.001 # 1% mutation rate per generation

### Main Evolution (Training) Loop

In [ ]:
pop_fit = []
pop = spawn_population(pop_size=population_size) #initial population
for gen in range(num_generations):
    # trainning
    pop, avg_fit = evaluate_population(pop)
    pop_fit.append(avg_fit) #record population average fitness
    new_pop = seed_next_population(pop, pop_size=population_size, mut_rate=mutation_rate)
    pop = new_pop

In [ ]:
plt.plot(pop_fit)

In [ ]:
avg_loss = 0
for i in range(len(pop)):
    x,y = next(iter(train_loader))
    pred = model(x.reshape(batch_size,784),pop[i].param)
    loss = loss_fn(pred,y)
    avg_loss += loss
avg_loss /= len(pop)
print(avg_loss)

Avg Loss new pop: 2.3336
Avg Loss after 10 gens: 2.3435

## Train with gradient-descent (comparison)

In [ ]:
p = torch.randn(784,10, requires_grad=True)
optim = torch.optim.Adam(lr=0.1, params=[p])

In [ ]:
loss_list = []
for i in range(50):
    for x,y in train_loader:
        optim.zero_grad()
        pred = model(x.reshape(batch_size,784),p)
        loss = loss_fn(pred,y)
        loss_list.append(loss)
        loss.backward()
        optim.step()
    print(loss)
plt.plot(loss_list)